# Scenario 05: Declarative Agents and Workflows

**Estimated Time**: 30 minutes

## Learning Objectives
- Define agents using YAML configuration
- Configure workflows declaratively
- Load and validate YAML schemas with Pydantic
- Switch between imperative and declarative patterns

## Prerequisites
- Completed Scenario 04 (Deterministic Workflows)
- Basic understanding of YAML syntax

In [ ]:
# Setup and imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.common.yaml_loader import (
    YAMLLoader,
    AgentConfig,
    WorkflowConfig,
    validate_agent_yaml,
    validate_workflow_yaml,
)
from src.agents.declarative import (
    DeclarativeAgent,
    DeclarativeAgentLoader,
    DeclarativeWorkflowLoader,
    load_agents_from_config,
    load_workflows_from_config,
)

print("✅ Imports successful")

## Part 1: Understanding Agent Configuration Schema

Agent configurations follow a Pydantic-validated schema with the following fields:

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `name` | string | Yes | Unique identifier (lowercase, underscores) |
| `model` | object | Yes | LLM configuration |
| `instructions` | string | Yes | System prompt |
| `tools` | list | No | Tool names to enable |
| `max_tokens` | int | No | Max response tokens (default: 4096) |

In [ ]:
# Example agent configuration as YAML string
agent_yaml = """
name: example_agent
model:
  provider: azure_openai
  deployment: gpt-4o
  temperature: 0.7
instructions: |
  You are a helpful assistant.
  Be concise and accurate.
tools:
  - search_web
max_tokens: 2048
"""

# Validate the YAML
config = validate_agent_yaml(agent_yaml)
print(f"Agent name: {config.name}")
print(f"Model: {config.model.provider}/{config.model.deployment}")
print(f"Temperature: {config.model.temperature}")
print(f"Tools: {config.tools}")

In [ ]:
# Validation catches errors
invalid_yaml = """
name: INVALID_NAME  # Must be lowercase
model:
  provider: unknown_provider  # Must be azure_openai or openai
  deployment: gpt-4
instructions: short  # Must be at least 10 characters
"""

try:
    validate_agent_yaml(invalid_yaml)
except Exception as e:
    print(f"Validation error (expected): {type(e).__name__}")
    print(f"Message: {e}")

## Part 2: Loading Agents from Files

The `DeclarativeAgentLoader` loads agent configurations from YAML files in a directory.

Directory structure:
```
configs/
  agents/
    research_agent.yaml
    summarizer_agent.yaml
```

In [ ]:
# Load individual agent
loader = YAMLLoader(base_path=project_root)

# Load research agent config
research_config = loader.load_agent("configs/agents/research_agent.yaml")
print(f"Loaded: {research_config.name}")
print(f"Instructions preview: {research_config.instructions[:100]}...")
print(f"Tools: {research_config.tools}")

In [ ]:
# Load all agents from directory
agent_loader = DeclarativeAgentLoader(
    agents_dir=project_root / "configs" / "agents"
)
agents = agent_loader.load_all()

print(f"Loaded {len(agents)} agents:")
for name, agent in agents.items():
    print(f"  - {name}: {agent.model_name} (temp={agent.temperature})")

In [ ]:
# Use a loaded agent
research_agent = agent_loader.get_agent("research_agent")
if research_agent:
    print(f"Agent: {research_agent}")
    
    # Run with mock (no real LLM call)
    import asyncio
    response = asyncio.get_event_loop().run_until_complete(
        research_agent.run("What are the latest trends in AI?")
    )
    print(f"Response: {response}")

## Part 3: Workflow Configuration Schema

Workflows are also defined declaratively:

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `name` | string | Yes | Workflow identifier |
| `description` | string | No | Human-readable description |
| `steps` | list | Yes | Ordered workflow steps |
| `on_error` | object | No | Error handling strategy |

In [ ]:
# Example workflow configuration
workflow_yaml = """
name: simple_pipeline
description: A simple two-step pipeline

steps:
  - name: research
    agent: research_agent
    prompt: "Research this topic: {topic}"
    outputs:
      - findings

  - name: summarize
    agent: summarizer_agent
    prompt: "Summarize: {findings}"
    outputs:
      - summary

on_error:
  strategy: retry
"""

workflow_config = validate_workflow_yaml(workflow_yaml)
print(f"Workflow: {workflow_config.name}")
print(f"Steps: {[s.name for s in workflow_config.steps]}")
print(f"Error strategy: {workflow_config.on_error.strategy}")

In [ ]:
# Load workflow from file
pipeline_config = loader.load_workflow(
    "configs/workflows/research_pipeline.yaml"
)

print(f"Pipeline: {pipeline_config.name}")
print(f"Description: {pipeline_config.description}")
print(f"\nSteps:")
for step in pipeline_config.steps:
    print(f"  {step.name}:")
    print(f"    agent: {step.agent}")
    print(f"    outputs: {step.outputs}")
    if step.retry:
        print(f"    retry: {step.retry.max_attempts} attempts")

## Part 4: Loading Complete Workflows

The `DeclarativeWorkflowLoader` combines agent and workflow loading to create
fully functional workflow engines.

In [ ]:
# Load agents first
agent_loader = DeclarativeAgentLoader(
    agents_dir=project_root / "configs" / "agents"
)
agent_loader.load_all()

# Then load workflows with agent reference
workflow_loader = DeclarativeWorkflowLoader(
    workflows_dir=project_root / "configs" / "workflows",
    agent_loader=agent_loader,
)
workflows = workflow_loader.load_all()

print(f"Loaded {len(workflows)} workflows:")
for name, engine in workflows.items():
    print(f"  - {name}: {len(engine.steps)} steps")

In [ ]:
# Convenience functions for one-liner loading
all_agents = load_agents_from_config(
    agents_dir=project_root / "configs" / "agents"
)
print(f"Loaded agents: {list(all_agents.keys())}")

all_workflows = load_workflows_from_config(
    workflows_dir=project_root / "configs" / "workflows",
    agents_dir=project_root / "configs" / "agents",
)
print(f"Loaded workflows: {list(all_workflows.keys())}")

## Part 5: Runtime Behavior Modification

One of the key benefits of declarative configuration is modifying behavior
without code changes.

In [ ]:
# Create a temporary modified config
modified_agent_yaml = """
name: modified_research_agent
model:
  provider: azure_openai
  deployment: gpt-4o
  temperature: 0.1  # Lower temperature for more consistent output
instructions: |
  You are a STRICT research assistant.
  
  RULES:
  - Only cite peer-reviewed sources
  - Include publication dates for all sources
  - Rate confidence level (high/medium/low) for each finding
  - Maximum 5 bullet points per source
tools:
  - search_academic
  - verify_source
max_tokens: 3000
"""

modified_config = validate_agent_yaml(modified_agent_yaml)
modified_agent = DeclarativeAgent(modified_config)

print(f"Modified agent: {modified_agent.name}")
print(f"Temperature: {modified_agent.temperature}")
print(f"Tools: {modified_agent.tools}")

In [ ]:
# Compare original vs modified
original = agent_loader.get_agent("research_agent")
modified = modified_agent

print("Configuration Comparison:")
print(f"{'Field':<20} {'Original':<25} {'Modified':<25}")
print("-" * 70)
print(f"{'Temperature':<20} {original.temperature:<25} {modified.temperature:<25}")
print(f"{'Max Tokens':<20} {original.max_tokens:<25} {modified.max_tokens:<25}")
print(f"{'Tools':<20} {str(original.tools):<25} {str(modified.tools):<25}")

## Part 6: Validation and Error Handling

Pydantic validation ensures configurations are correct before runtime.

In [ ]:
# Various validation scenarios
test_cases = [
    # Missing required field
    ("""
name: test_agent
model:
  provider: azure_openai
  deployment: gpt-4
# Missing instructions!
""", "Missing 'instructions' field"),
    
    # Invalid name pattern
    ("""
name: 123_invalid
model:
  provider: azure_openai
  deployment: gpt-4
instructions: This is a test agent.
""", "Invalid name pattern (must start with letter)"),
    
    # Temperature out of range
    ("""
name: test_agent
model:
  provider: azure_openai
  deployment: gpt-4
  temperature: 5.0  # Max is 2.0
instructions: This is a test agent.
""", "Temperature out of range"),
]

for yaml_content, description in test_cases:
    try:
        validate_agent_yaml(yaml_content)
        print(f"❌ {description}: Should have failed!")
    except Exception as e:
        print(f"✅ {description}: Caught correctly")

## Part 7: Best Practices

### Configuration Organization

```
configs/
  agents/
    research_agent.yaml
    summarizer_agent.yaml
    validator_agent.yaml
  workflows/
    research_pipeline.yaml
    content_pipeline.yaml
  environments/
    dev.yaml        # Development overrides
    prod.yaml       # Production settings
```

### Naming Conventions

- Use `snake_case` for names
- Be descriptive: `research_agent` not `ra`
- Include purpose in workflow names: `content_review_pipeline`

### Version Control

- Track all config changes in git
- Use meaningful commit messages for config changes
- Consider config file validation in CI/CD

In [ ]:
# Example: Config validation function for CI/CD
from pathlib import Path

def validate_all_configs(base_path: Path) -> list[str]:
    """Validate all configuration files."""
    errors = []
    loader = YAMLLoader(base_path)
    
    # Validate agents
    agents_dir = base_path / "configs" / "agents"
    if agents_dir.exists():
        for file in agents_dir.glob("*.yaml"):
            try:
                loader.load_agent(file)
                print(f"  ✅ {file.name}")
            except Exception as e:
                errors.append(f"{file.name}: {e}")
                print(f"  ❌ {file.name}: {e}")
    
    # Validate workflows
    workflows_dir = base_path / "configs" / "workflows"
    if workflows_dir.exists():
        for file in workflows_dir.glob("*.yaml"):
            try:
                loader.load_workflow(file)
                print(f"  ✅ {file.name}")
            except Exception as e:
                errors.append(f"{file.name}: {e}")
                print(f"  ❌ {file.name}: {e}")
    
    return errors

print("Config Validation:")
validation_errors = validate_all_configs(project_root)
print(f"\nTotal errors: {len(validation_errors)}")

## 🎯 Exercise: Add a New Agent via Configuration

Create a new agent configuration for a "fact_checker_agent" that:

1. Uses GPT-4o with low temperature (0.2) for consistent results
2. Has instructions focused on verifying claims
3. Uses tools: `search_web`, `check_source_reliability`
4. Limits responses to 1500 tokens

### Your Task:

1. Write the YAML configuration below
2. Validate it using `validate_agent_yaml()`
3. Create a `DeclarativeAgent` from it

In [ ]:
# Your solution here
fact_checker_yaml = """
# TODO: Add your agent configuration
name: fact_checker_agent
model:
  provider: azure_openai
  deployment: gpt-4o
  temperature: 0.2
instructions: |
  You are a fact-checking specialist.
  
  Your responsibilities:
  1. Verify claims against reliable sources
  2. Rate claim accuracy (True/False/Partially True/Unverifiable)
  3. Provide evidence for your assessment
  4. Note any important context or caveats
tools:
  - search_web
  - check_source_reliability
max_tokens: 1500
"""

# Validate your configuration
try:
    fact_checker_config = validate_agent_yaml(fact_checker_yaml)
    print(f"✅ Valid configuration for: {fact_checker_config.name}")
    
    # Create agent
    fact_checker = DeclarativeAgent(fact_checker_config)
    print(f"Agent created: {fact_checker}")
except Exception as e:
    print(f"❌ Validation failed: {e}")

## Summary

In this scenario, you learned:

1. **Agent Configuration Schema**: Define agents with name, model, instructions, tools
2. **YAML Validation**: Pydantic ensures configs are valid before runtime
3. **Loading Agents**: Use `DeclarativeAgentLoader` to load from files
4. **Workflow Configuration**: Define multi-step workflows declaratively
5. **Runtime Modification**: Change behavior without code changes
6. **Best Practices**: Organization, naming, version control

### Next Steps

- **Scenario 6**: Moderating Agent Discussions
- Explore environment-specific configuration overrides
- Build a configuration management UI